# Intro to FAST data

> Abstract: Some datasets are available more rapidly than others (for space weather monitoring). These are delivered through a special "FAST" processing chain, as opposed to the normal operational "OPER" chain. They are available under different collection names containing the string "FAST". Here we compare the availability and quality of FAST data in contrast to OPER data.

NB: Data is currently only available on the DISC machine for select users

In [ ]:
SERVER_URL = 'https://vires.services/ows'

In [ ]:
%load_ext watermark
%watermark -i -v -p viresclient,pandas,xarray,matplotlib,tqdm

In [ ]:
from viresclient import SwarmRequest
import datetime as dt
from tqdm import tqdm
import matplotlib as mpl
import matplotlib.pyplot as plt
from operator import or_
from functools import reduce

## Which data are available FAST?

In [ ]:
request = SwarmRequest(SERVER_URL)

all_collections = request.available_collections(details=False)
# Identify collections with FAST in the name
collections = {group: [c for c in colls if "FAST" in c] for (group, colls) in all_collections.items()}
# Filter the empty groups
collections = {group: colls for group, colls in collections.items() if len(colls) > 0}
collections


Let's use `'SW_FAST_MAGA_LR_1B'` as an example and compare to `'SW_OPER_MAGA_LR_1B'`.

We can query the availability times of the underlying products:

In [ ]:
fast_availability = request.available_times("SW_FAST_MAGA_LR_1B")
fast_availability

**Note: VirES only keeps a rolling store of the most recent month of FAST data.**

In [ ]:
oper_availability = request.available_times("SW_OPER_MAGA_LR_1B")
oper_availability

In [ ]:
print(f"Latest OPER: {oper_availability['endtime'].iloc[-1]}")
print(f"Latest FAST: {fast_availability['endtime'].iloc[-1]}")

As you can see above, `OPER` data is delivered in full-day chunks with a few days delay. `FAST` data are typically delivered in several-hour chunks and available as soon as possible, subject to downlink opportunities from the satellite passes over the ground stations and subsequent processing time.

## Accessing FAST data

`FAST` data are identical in format to `OPER` data so can be accessed and used in the same way through VirES.

In the following example, we access the latest available day of `OPER` data and compare to the available `FAST` data, for each of Swarm A, B, C.

In [ ]:
collection_root = "SW_{}_MAG{}_LR_1B"

def find_latest_oper(spacecraft="A"):
    """Identify the latest availability time for OPER data"""
    collection = collection_root.format("OPER", spacecraft)
    request = SwarmRequest(SERVER_URL)
    times = request.available_times(collection)
    return times['endtime'].iloc[-1].to_pydatetime()

def fetch_data(spacecraft="A", type="OPER", start=None, end=None):
    """Fetch either OPER or FAST data"""
    collection = collection_root.format(type, spacecraft)
    request = SwarmRequest(SERVER_URL)
    request.set_collection(collection)
    request.set_products(
        measurements=["B_NEC", "Flags_B"],
        models=["CHAOS"],
    )
    data = request.get_between(start, end, asynchronous=False, show_progress=False)
    ds = data.as_xarray()
    ds["B_NEC_res_CHAOS"] = ds["B_NEC"] - ds["B_NEC_CHAOS"]
    return ds

In [ ]:
# Find the latest availability times of OPER data per-satellite
oper_end_times = {sc: find_latest_oper(sc) for sc in "ABC"}
oper_end_times

In [ ]:
data_fast = {}
data_oper = {}
for sc in tqdm("ABC"):
    # Fetch the latest available day of OPER data
    oper_end = oper_end_times[sc] + dt.timedelta(seconds=1)
    oper_start = oper_end - dt.timedelta(days=1)
    data_oper[sc] = fetch_data(sc, "OPER", oper_start, oper_end)
    # Fetch all FAST data from then until now
    data_fast[sc] = fetch_data(sc, "FAST", oper_end, dt.datetime.now())

In [ ]:
def flag_filter(ds):
    """A simplistic filter for close-to-nominal data"""
    # Filtering by Flags_B
    # For flag meanings, see https://swarmhandbook.earth.esa.int/catalogue/SW_MAGx_LR_1B
    # This includes Charlie data, where the ASM was lost
    nominal = 0b0000
    asm_off = 0b0001
    vfm_asm_discrepency = 0b1000
    bitmask_filters = (nominal, asm_off, vfm_asm_discrepency)
    flags = ds["Flags_B"]
    flags_masked = reduce(or_, [flags & x for x in bitmask_filters])
    return ds.where(flags == flags_masked)

In [ ]:
# Identify the minimal and maximal times in the datasets
tmin = min(data_oper[sc]["Timestamp"].data[0] for sc in "ABC")
tmax = max(data_fast[sc]["Timestamp"].data[-1] for sc in "ABC")

fig, axes = plt.subplots(nrows=3, figsize=(15, 10), sharex=True)
for ax, sc in zip(axes, "ABC"):
    # Remove bad data before plotting
    ds_oper_nominal = flag_filter(data_oper[sc])
    ds_fast_nominal = flag_filter(data_fast[sc])
    ax.set_prop_cycle("color", ["tab:blue", "tab:orange", "tab:green"])
    ds_oper_nominal["B_NEC_res_CHAOS"].plot.line(x="Timestamp", ax=ax)
    ds_fast_nominal["B_NEC_res_CHAOS"].plot.line(x="Timestamp", ax=ax)
    ax.set_xlim(tmin, tmax)
    # Add shading behind OPER data section
    oper_start = data_oper[sc]["Timestamp"].data[0]
    oper_end = data_oper[sc]["Timestamp"].data[-1]
    ymin, ymax = ax.get_ylim()
    ax.fill_betweenx((ymin, ymax), oper_start, oper_end, alpha=0.3, color="grey")
    # Add and fix some labelling
    ax.text(oper_end, ymax, "⬅️ OPER    FAST ➡️", ha="center", va="top")
    ax.set_xlim(oper_start, ax.get_xlim()[-1])
    ax.set_xlabel("")
    ax.set_ylabel(f"Swarm {sc}\n{ax.get_ylabel()}")

ax.xaxis.set_major_formatter(mpl.dates.DateFormatter("%Y-%m-%d\n%H:%M"))